In [14]:
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
)
from llama_index.core.query_engine.pandas import PandasInstructionParser
from llama_index.llms.openai import OpenAI
from llama_index.core import PromptTemplate
from llama_index.llms.ollama import Ollama

In [26]:
import phoenix as px
px.close_app()
px.launch_app()
import llama_index.core
llama_index.core.set_global_handler("arize_phoenix")
print(px.active_session())

INFO:phoenix.datasets.dataset:Dataset: phoenix_dataset_a8c673da-e85b-49ab-8f4a-7351432662f1 initialized
Dataset: phoenix_dataset_a8c673da-e85b-49ab-8f4a-7351432662f1 initialized
No active session to close
🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [27]:
import pandas as pd
df = pd.read_csv(r"D:\Gen_AI_Tutorials\langchain\llamaindexrag\csv_data\titanic_train.csv")

In [60]:
instruction_str = (
    "1. Convert the query to Python code using Pandas.\n"
    # "2. The final line of code should be a Python expression. \n"
    "2. The code should represent a solution to the query.\n"
    "3. PRINT ONLY THE EXPRESSION.\n"
    "4. Do not quote the expression.\n"
)

pandas_prompt_str = (
    "You are working with a pandas dataframe in Python.\n"
    "The name of the dataframe is `df`.\n"
    "This is the result of `df.head()`:\n"
    "{df_str}\n\n"
    "Follow these instructions:\n"
    "{instruction_str}\n"
    "Query: {query_str}\n\n"
    "Expression:"
)
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n\n"
    "Pandas Instructions (optional):\n{pandas_instructions}\n\n"
    "Pandas Output: {pandas_output}\n\n"
    "Response: "
)

pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str, df_str=df.head(5)
)
pandas_output_parser = PandasInstructionParser(df)
response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)

In [61]:
# instruction_str = (
#     "1. Convert the query to executable Python code using Pandas.\n"
#     "2. The final line of code should be a Python expression that can be called with the `eval()` function.\n"
#     "3. The code should represent a solution to the query.\n"
#     "4. PRINT ONLY THE EXPRESSION.\n"
#     "5. Do not quote the expression.\n"
# )

# pandas_prompt_str = (
#     "You are working with a pandas dataframe in Python.\n"
#     "The name of the dataframe is `df`.\n"
#     "This is the result of `df.head()`:\n"
#     "{df_str}\n\n"
#     "Follow these instructions:\n"
#     "{instruction_str}\n"
#     "Query: {query_str}\n\n"
#     "Expression:"
# )
# response_synthesis_prompt_str = (
#     "Given an input question, synthesize a response from the query results.\n"
#     "Query: {query_str}\n\n"
#     "Pandas Instructions (optional):\n{pandas_instructions}\n\n"
#     "Pandas Output: {pandas_output}\n\n"
#     "Response: "
# )

# pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
#     instruction_str=instruction_str, df_str=df.head(5)
# )
# pandas_output_parser = PandasInstructionParser(df)
# response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)

In [62]:
llm= Ollama(model="phi",request_timeout=300)

In [75]:
qp = QP(
    modules={
        "input": InputComponent(),
        "pandas_prompt": pandas_prompt,
        "llm1": llm,
        "pandas_output_parser": pandas_output_parser,
        "response_synthesis_prompt": response_synthesis_prompt,
        "llm2": llm,
    },
    verbose=True,
)
qp.add_chain(["input", "pandas_prompt", "llm1", "pandas_output_parser"])
qp.add_links(
    [
        Link("input", "response_synthesis_prompt", dest_key="query_str"),
        Link(
            "llm1", "response_synthesis_prompt", dest_key="pandas_instructions"
        ),
        Link(
            "pandas_output_parser",
            "response_synthesis_prompt",
            dest_key="pandas_output",
        ),
    ]
)
# add link from response synthesis prompt to llm2
qp.add_link("response_synthesis_prompt", "llm2")

In [76]:
llm

{'model': 'phi',
 'created_at': '2024-04-06T19:34:41.6010648Z',
 'message': {'role': 'assistant', 'content': ''},
 'done': True}

In [77]:
response = qp.run(
    query_str="What is the correlation between survival and age?",
)

AttributeError: 'dict' object has no attribute 'set_callback_manager'

In [19]:
import logging
import sys
from IPython.display import Markdown, display

import pandas as pd
from llama_index.core.query_engine import PandasQueryEngine


logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [20]:
llm= Ollama(model="phi",request_timeout=300)

In [21]:

# Test on some sample data
df = pd.DataFrame(
    {
        "city": ["Toronto", "Tokyo", "Berlin"],
        "population": [2930000, 13960000, 3645000],
    }
)

In [22]:
query_engine = PandasQueryEngine(df=df, verbose=True, llm=llm)

In [23]:
response = query_engine.query(
    "What is the city with the highest population?",
)

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
> Pandas Instructions:
```
 The city with the highest population can be found using the `max()` function and passing it a lambda function that returns the first element of each row in the dataframe, which represents the population value. Then we use the `idxmax()` method to get the index of the row with the maximum population. Finally, we access the city name from this index using the `loc[]` method.

```
city_with_highest_population = df.iloc[df['population'].apply(lambda x: int(x)).idxmax()]['city']
print(city_with_highest_population)
```

```
> Pandas Output: There was an error running the output as Python code. Error message: invalid syntax (<string>, line 0)


Traceback (most recent call last):
  File "d:\Gen_AI_Tutorials\langchain\venv\lib\site-packages\llama_index\core\query_engine\pandas\output_parser.py", line 54, in default_output_processor
    output_str = str(safe_eval(module_end_str, global_vars, local_vars))
  File "d:\Gen_AI_Tutorials\langchain\venv\lib\site-packages\llama_index\core\exec_utils.py", line 158, in safe_eval
    return eval(__source, _get_restricted_globals(__globals), __locals)
  File "<string>", line 0
    
SyntaxError: invalid syntax


In [66]:
pip show Ollama

Name: ollama
Version: 0.1.8
Summary: The official Python client for Ollama.
Home-page: https://ollama.ai
Author: Ollama
Author-email: hello@ollama.com
License: MIT
Location: d:\gen_ai_tutorials\langchain\venv\lib\site-packages
Requires: httpx
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [69]:
import ollama

In [73]:
llm = ollama.chat(model='phi')

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


In [74]:
llm

{'model': 'phi',
 'created_at': '2024-04-06T19:34:41.6010648Z',
 'message': {'role': 'assistant', 'content': ''},
 'done': True}